In [ ]:
!pip install -q kaggle

In [ ]:
!nvidia-smi

Sat Jun  5 02:27:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sameershaikzz","key":"5a667e30681e172c5f97364076a604c7"}'}

In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c turnkey-summer-interns-idl-hw1p2

  0% 0.00/508k [00:00<?, ?B/s]
100% 508k/508k [00:00<00:00, 32.6MB/s]
  0% 0.00/500k [00:00<?, ?B/s]
100% 500k/500k [00:00<00:00, 68.0MB/s]
 97% 301M/310M [00:08<00:00, 35.8MB/s]
100% 310M/310M [00:08<00:00, 37.7MB/s]
100% 3.52G/3.52G [01:36<00:00, 43.9MB/s]
100% 3.52G/3.52G [01:36<00:00, 39.2MB/s]
  0% 0.00/518 [00:00<?, ?B/s]
100% 518/518 [00:00<00:00, 426kB/s]
 97% 5.00M/5.16M [00:00<00:00, 9.43MB/s]
100% 5.16M/5.16M [00:00<00:00, 9.58MB/s]
 98% 316M/322M [00:11<00:00, 26.9MB/s]
100% 322M/322M [00:11<00:00, 28.4MB/s]
  0% 0.00/3.43M [00:00<?, ?B/s]
100% 3.43M/3.43M [00:00<00:00, 114MB/s]


In [ ]:
!unzip \*.zip  -d /content/drive/MyDrive/Homeworks/data

Archive:  sample.csv.zip
  inflating: /content/drive/MyDrive/Homeworks/data/sample.csv  

Archive:  dev.npy.zip
  inflating: /content/drive/MyDrive/Homeworks/data/dev.npy  

Archive:  test_labels.npy.zip
  inflating: /content/drive/MyDrive/Homeworks/data/test_labels.npy  

Archive:  train_labels.npy.zip
  inflating: /content/drive/MyDrive/Homeworks/data/train_labels.npy  

Archive:  train.npy.zip
  inflating: /content/drive/MyDrive/Homeworks/data/train.npy  

Archive:  dev_labels.npy.zip
  inflating: /content/drive/MyDrive/Homeworks/data/dev_labels.npy  

Archive:  test.npy.zip
  inflating: /content/drive/MyDrive/Homeworks/data/test.npy  

7 archives were successfully processed.


In [ ]:
#!unzip \*.zip  && rm *.zip

Archive:  test_labels.npy.zip
replace test_labels.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test_labels.npy         

Archive:  dev.npy.zip
  inflating: dev.npy                 

Archive:  train_labels.npy.zip
  inflating: train_labels.npy        

Archive:  train.npy.zip
  inflating: train.npy               

A


Archive:  test.npy.zip
  inflating: test.npy                

Archive:  dev_labels.npy.zip
  inflating: dev_labels.npy          

Archive:  sample.csv.zip
  inflating: sample.csv              

7 archives were successfully processed.


In [ ]:
import numpy as np
import torch


In [ ]:
import torch.nn as nn
import time
import os

In [ ]:
# loading data 

#train_data=np.load("train.npy", allow_pickle=True)
dev_data=np.load("/content/drive/MyDrive/Homeworks/data/dev.npy", allow_pickle=True)
test_data=np.load("/content/drive/MyDrive/Homeworks/data/test.npy" , allow_pickle=True)

#train_labels=np.load("train_labels.npy", allow_pickle=True)
dev_labels=np.load("/content/drive/MyDrive/Homeworks/data/dev_labels.npy", allow_pickle=True)
test_labels=np.load("/content/drive/MyDrive/Homeworks/data/test_labels.npy", allow_pickle=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data=np.load("/content/drive/MyDrive/Homeworks/data/train.npy", allow_pickle=True)
train_labels=np.load("/content/drive/MyDrive/Homeworks/data/train_labels.npy", allow_pickle=True)

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y, context):
        # Store paramters as class variables
        self.X=X
        self.Y=Y
        self.context=context
        # Taking the data in variable x and arraning all the frames in one list
        tot_data=[]
        for x in X:
          for i in x:
            tot_data.append(i)
        
        self.X=np.array(tot_data,dtype=np.float)
     
        self.X=np.pad(self.X,pad_width=((self.context,self.context),(0,0)),mode="constant",constant_values=0)
        tot_data=[]
        for y in Y:
          for j in y:
            tot_data.append(j)
        self.Y=np.array(tot_data,dtype=np.float)
        # print("self.Y at index 0 istype is:",self.Y[0])
        
         
            
        
    def __len__(self):
        # print(self.Y.shape[0])
        return len(self.Y)

    def __getitem__(self,index):

        
        start_index=index
        end_index=index+2*self.context+1
        xx=self.X[start_index:end_index,:].flatten()
        # print(xx)
        # xx=self.X[index]
        yy=self.Y[index]
        return xx,yy
    def collate_fn(batch):
        
        ### Select all data from batch (1 line)
        batch_x = [x for x,y in batch]
        # print(batch_x)
        ### Select all labels from batch (1 line)
        batch_y = [y for x,y in batch]
        
        ### Convert batched data and labels to tensors (2 lines)
        batch_x = torch.as_tensor(batch_x)
        batch_y = torch.as_tensor(batch_y)
        
        ### Return batched data and labels (1 line)
        return batch_x, batch_y


In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context=15
dataset=MyDataset(train_data,train_labels,context)
train_loader_args = dict(shuffle=True, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
train_loader=torch.utils.data.DataLoader(dataset,**train_loader_args)

In [ ]:
import torch.nn as nn
class Simple_MLP(nn.Module):
    def __init__(self,input_output_data):
        super(Simple_MLP, self).__init__()
        self.input_output_data=input_output_data
        layers=[]
        for i in range(len(self.input_output_data)-2):
          layers.append(nn.Linear(input_output_data[i],input_output_data[i+1]))
          layers.append(nn.BatchNorm1d(input_output_data[i+1]))
          layers.append(nn.ReLU())
          layers.append(nn.Dropout(p=0.1))
        layers.append(nn.Linear(input_output_data[-2],input_output_data[-1]))
        
        self.net=nn.Sequential(*layers)

    def forward(self, x):
        # make sure to return the output after 
        # call the network created above
        return self.net(x.float())

In [ ]:
#print(context)
cuda = torch.cuda.is_available()


In [ ]:
import torch.optim as optim
context=15
k=(context*2+1)*40
model=Simple_MLP([k,4096,3072,2048,1024,1024,512,256,71])
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters())  # model.parameters refer to the network’s weights
device=torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=1240, out_features=4096, bias=True)
    (1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=4096, out_features=3072, bias=True)
    (5): BatchNorm1d(3072, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=3072, out_features=2048, bias=True)
    (9): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=2048, out_features=1024, bias=True)
    (13): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.1, inplace=False)
    (16): Linear(in_features=1024, out_features=1024, bias=True)
    (17): BatchNorm1d(1024, eps=1e-05, momentum=0.1, aff

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):

  print("Training....") 
  model.train()

  running_loss=0.0
  total_predictions=0.0 
  correct_predictions=0.0
  start_time=time.time()

  for batch_idx, (data, target) in enumerate(train_loader):
    print(batch_idx)
    optimizer.zero_grad() 
    data=data.to(device)
  
    target=target.to(device) 

    outputs=model(data) 
    loss=criterion(outputs, target.long())
    running_loss+=loss.item()

    loss.backward()    # we calculate the gradients by calling the backward() function
    optimizer.step()  # is used to update the weights using the calculated gradients.
    
    
    # Here the accuracy can be calculaated with the no training examples being 
    # correclty predicted i,e target is the y and outputs is y^)
    

    value,index=torch.max(outputs,1)
    correct_predictions=torch.sum(index==target)
    total_predictions=outputs.shape[0]
    
    end_time=time.time()
    
    running_loss/=len(train_loader)
    
    print("Training Loss:", running_loss, "Time: ",end_time-start_time,'s')
    
    acc=(correct_predictions/total_predictions)*100.0
    
    #path = "/content/drive/MyDrive/Homeworks/model"
    #torch.save(model.state_dict(), os.path.join(path, 'batch_idx'))
    #model_save_name = 'classifier.pt'
    #path = '/content/drive/MyDrive/Homeworks/model/classifier.pt' 
    #torch.save(model.state_dict(), path)


    
    print('Training Accuracy:', acc, "% ") 
    print("=====================================================================")
  return running_loss ,acc

In [ ]:
for i in range(3):
  train_epoch(model, train_loader, criterion, optimizer)
  #val_epoch(model, val_loader, criterion)
  path='/content/drive/MyDrive/Homeworks/model/model_drp01_4096_2time'+str(i)
  torch.save(model.state_dict(), path)

Streaming output truncated to the last 5000 lines.
70950
Training Loss: 6.78682917255615e-06 Time:  1239.8406023979187 s
Training Accuracy: tensor(84.3750, device='cuda:0') % 
70951
Training Loss: 7.752208881512725e-06 Time:  1239.8560163974762 s
Training Accuracy: tensor(82.4219, device='cuda:0') % 
70952
Training Loss: 6.844198737186476e-06 Time:  1239.8740766048431 s
Training Accuracy: tensor(82.8125, device='cuda:0') % 
70953
Training Loss: 5.781266519733216e-06 Time:  1239.8901863098145 s
Training Accuracy: tensor(86.7188, device='cuda:0') % 
70954
Training Loss: 6.750119106886093e-06 Time:  1239.9082834720612 s
Training Accuracy: tensor(83.5938, device='cuda:0') % 
70955
Training Loss: 7.530304903639223e-06 Time:  1239.9256503582 s
Training Accuracy: tensor(82.8125, device='cuda:0') % 
70956
Training Loss: 6.538582754227523e-06 Time:  1239.9426536560059 s
Training Accuracy: tensor(85.5469, device='cuda:0') % 
70957
Training Loss: 6.4131990687794315e-06 Time:  1239.9631724357605 s

In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context=15
val_dataset=MyDataset(dev_data,dev_labels,context)
val_loader_args = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=64)
val_loader=torch.utils.data.DataLoader(val_dataset,**val_loader_args)



In [ ]:
def val_epoch(model, val_loader, criterion):
  print("validating....") 
  model.eval()

  running_loss=0.0
  total_predictions=0.0 
  correct_predictions=0.0
  start_time=time.time()
  for batch_idx, (data, target) in enumerate(val_loader):
    print(batch_idx)
 
    data=data.to(device)

    target=target.to(device) #data & sodel on same device

    outputs=model(data) 
    loss=criterion(outputs, target.long())
    running_loss+=loss.item()

    
    
    
    # Here the accuracy can be calculaated with the no training examples being 
    # correclty predicted i,e target is the y and outputs is y^)


    value,index=torch.max(outputs,1)
    correct_predictions=torch.sum(index==target)
    total_predictions=outputs.shape[0]
    # print(value,index,"--------------------")

    # print(outputs[1],target[1],"===============================","index is :",value)
    # if index==target[1]:
    #   print(index,target[1])
    # # ,target[1])

    
    end_time=time.time()

    running_loss/=len(val_loader)

    print("testing Loss:", running_loss, "Time: ",end_time-start_time,'s')
    
    acc=(correct_predictions/total_predictions)*100.0
   

    print('Testing Accuracy:', acc, "% ") 
  return running_loss ,acc
    
    

In [ ]:
val_epoch(model, val_loader, criterion)

NameError: ignored

In [ ]:
"""
for i in range(1):
  train_epoch(model, train_loader, criterion, optimizer)
  #val_epoch(model, val_loader, criterion)
  path='/content/drive/MyDrive/Homeworks/model/epoch_222'+str(i)
  torch.save(model.state_dict(), path)
"""


Streaming output truncated to the last 5000 lines.
70950
Training Loss: 7.216830854425211e-06 Time:  992.7240068912506 s
Training Accuracy: tensor(82.0312, device='cuda:0') % 
70951
Training Loss: 8.776608165884018e-06 Time:  992.7425715923309 s
Training Accuracy: tensor(81.2500, device='cuda:0') % 
70952
Training Loss: 9.637080605217416e-06 Time:  992.7565789222717 s
Training Accuracy: tensor(75., device='cuda:0') % 
70953
Training Loss: 7.968230164433222e-06 Time:  992.7704954147339 s
Training Accuracy: tensor(80.4688, device='cuda:0') % 
70954
Training Loss: 7.292762592491522e-06 Time:  992.783243894577 s
Training Accuracy: tensor(82.0312, device='cuda:0') % 
70955
Training Loss: 7.847214300095252e-06 Time:  992.796195268631 s
Training Accuracy: tensor(82.0312, device='cuda:0') % 
70956
Training Loss: 8.81196030488083e-06 Time:  992.8077445030212 s
Training Accuracy: tensor(77.7344, device='cuda:0') % 
70957
Training Loss: 9.478574842760618e-06 Time:  992.8208241462708 s
Training Ac

In [ ]:
import torch

In [ ]:
#net.load_state_dict(torch.load(PATH))
#net.eval()

In [ ]:
#model=torch.load('/content/drive/MyDrive/Homeworks/model/classifier.pt')
#print(model)

In [ ]:
#model.keys()

In [ ]:
test_data=np.load("/content/drive/MyDrive/Homeworks/data/test.npy" , allow_pickle=True)

In [ ]:
class MyDatasettest(torch.utils.data.Dataset):
    def __init__(self, X, context):
        # Store paramters as class variables
        self.X=X
       
        self.context=context
        # Taking the data in variable x and arraning all the frames in one list
        tot_data=[]
        for x in X:
          for i in x:
            tot_data.append(i)
        
        self.X=np.array(tot_data,dtype=np.float)
      #  padding the data at starting of the data and ending of  the data with context no of rows 
        self.length=self.X.shape[0]
        self.X=np.pad(self.X,pad_width=((self.context,self.context),(0,0)),mode="constant",constant_values=0)
    
            
        
    def __len__(self):
        # print(self.Y.shape[0])
        return self.length

    def __getitem__(self,index):

        
        start_index=index
        end_index=index+2*self.context+1
        xx=self.X[start_index:end_index,:].flatten()
        # print(xx)
        # xx=self.X[index]
        # yy=self.Y[index]
        return xx
    def collate_fn(batch):
        
        ### Select all data from batch (1 line)
        batch_x =batch
        # print(batch_x)
        ### Select all labels from batch (1 line)
        # batch_y = [y for x in batch]
        
        ### Convert batched data and labels to tensors (2 lines)
        batch_x = torch.as_tensor(batch_x)
        # batch_y = torch.as_tensor(batch_y)
        
        ### Return batched data and labels (1 line)
        return batch_x
        

In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context=15
dataset=MyDatasettest(test_data,context)
train_loader_args = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=64)
test_loader=torch.utils.data.DataLoader(dataset,**train_loader_args)


In [ ]:
import numpy as np

In [ ]:
model

Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=840, out_features=2048, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=1024, out_features=512, bias=True)
    (10): ReLU()
    (11): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): ReLU()
    (14): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Linear(in_features=512, out_features=128, bias=True)
  )
)

In [ ]:
#PATH="/content/drive/MyDrive/Homeworks/model/epoch_2221"
path="/content/drive/MyDrive/Homeworks/model/batch_idx"

In [ ]:
model=Simple_MLP([k,2048,1024,512,512,256,128,71])

In [ ]:
state_dict.keys()

odict_keys(['net.0.weight', 'net.0.bias', 'net.2.weight', 'net.2.bias', 'net.2.running_mean', 'net.2.running_var', 'net.2.num_batches_tracked', 'net.3.weight', 'net.3.bias', 'net.5.weight', 'net.5.bias', 'net.5.running_mean', 'net.5.running_var', 'net.5.num_batches_tracked', 'net.6.weight', 'net.6.bias', 'net.8.weight', 'net.8.bias', 'net.8.running_mean', 'net.8.running_var', 'net.8.num_batches_tracked', 'net.9.weight', 'net.9.bias', 'net.11.weight', 'net.11.bias', 'net.11.running_mean', 'net.11.running_var', 'net.11.num_batches_tracked', 'net.12.weight', 'net.12.bias', 'net.14.weight', 'net.14.bias', 'net.14.running_mean', 'net.14.running_var', 'net.14.num_batches_tracked', 'net.15.weight', 'net.15.bias', 'net.17.weight', 'net.17.bias', 'net.17.running_mean', 'net.17.running_var', 'net.17.num_batches_tracked', 'net.18.weight', 'net.18.bias'])

In [ ]:
state_dict = torch.load(path,map_location=torch.device('cpu'))
#print(len(state_dict["net.14.weight"]))
model.load_state_dict(state_dict)


256


<All keys matched successfully>

In [ ]:
print(state_dict.keys())

odict_keys(['net.0.weight', 'net.0.bias', 'net.2.weight', 'net.2.bias', 'net.2.running_mean', 'net.2.running_var', 'net.2.num_batches_tracked', 'net.3.weight', 'net.3.bias', 'net.5.weight', 'net.5.bias', 'net.5.running_mean', 'net.5.running_var', 'net.5.num_batches_tracked', 'net.6.weight', 'net.6.bias', 'net.8.weight', 'net.8.bias', 'net.8.running_mean', 'net.8.running_var', 'net.8.num_batches_tracked', 'net.9.weight', 'net.9.bias', 'net.11.weight', 'net.11.bias', 'net.11.running_mean', 'net.11.running_var', 'net.11.num_batches_tracked', 'net.12.weight', 'net.12.bias', 'net.14.weight', 'net.14.bias', 'net.14.running_mean', 'net.14.running_var', 'net.14.num_batches_tracked', 'net.15.weight', 'net.15.bias'])


In [ ]:
def test_epoch(model, test_loader):

  print("testing data is taken....") 
  model.eval()
  
  
  
  result=[]
  for batch_idx, data in enumerate(test_loader):
    data=data.to(device)
    outputs=model(data) 
    # print(outputs.shape)
    value,index=torch.max(outputs,1)
    result.append(index.tolist())
  return result  
    
  

    
 
result=test_epoch(model, test_loader)

testing data is taken....


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print(result[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 68, 68, 68, 68, 68, 68, 68, 40, 40, 40, 40, 40, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 40, 40, 40, 40, 40, 40, 40, 45, 45, 45, 45, 45, 45, 45, 0, 0, 0, 6, 6, 18, 6, 6, 25, 25, 25, 6, 31, 31, 31, 31, 31, 31, 31, 31, 31, 21, 21, 43, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 8, 36, 36, 8, 8, 8, 8, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 45, 45, 45, 45, 45, 18, 18, 18, 18, 18, 18, 18, 6, 6, 6, 15, 15, 15, 15, 15, 15, 15, 3, 15, 15, 15, 15, 46, 46, 43, 43, 43, 43, 43, 43, 43, 43, 8, 8, 8, 8, 8, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 0, 0, 0, 0, 0, 0, 0, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 25, 25, 25, 25, 25, 25, 25, 25, 25]


In [ ]:
import torch

In [ ]:
tensor_result=torch.as_tensor(result[0:-1]).flatten()
tensor_result.shape
last=torch.as_tensor(result[-1])
final_tensor=torch.hstack((tensor_result,last))
final=final_tensor.tolist()
from pandas import DataFrame
# your_list = ['item1', 'item2', 'item3',...]
df = DataFrame (final,columns=['label'])
df.to_csv("4096_2_256ba_drp01.csv",index_label='id')
df.head

<bound method NDFrame.head of          label
0            0
1            0
2            0
3            0
4            0
...        ...
1618830      0
1618831      0
1618832      0
1618833      0
1618834      0

[1618835 rows x 1 columns]>

In [ ]:
!kaggle competitions submit -c turnkey-summer-interns-idl-hw1p2 -f 4096_2_256ba_drp01.csv -m "Message"

100% 15.5M/15.5M [00:04<00:00, 3.37MB/s]
Successfully submitted to Turnkey Summer Interns IDL : HW1P2

In [ ]:
!kaggle competitions submit -c turnkey-summer-interns-idl-hw1p2 -f Drop_256ba_con15.csv -m "Message"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
